In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv
from scipy.special import jn_zeros
from scipy.optimize import minimize
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_4D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2), x[1])
    theta3 = np.arctan2(x[3], x[2])
    
    
    return rt+ndt, (theta1, theta2, theta3)

In [3]:
def series_bessel_fpt(t, a=1, sigma=1, nu=0, n=100):
    zeros = np.asarray([float(besseljzero(nu, i+1)) for i in range(n)])
    fpt = np.zeros(t.shape)
    
    for i in range(t.shape[0]):
        series = np.sum((zeros**(nu+1)/jv(nu+1, zeros)) * np.exp(-(zeros**2 * sigma**2)/(2*a**2)*t[i]))
        fpt[i] = sigma**2/(2**nu * a**2 * gamma(nu + 1)) * series
        
    return interp1d(t, fpt)

In [4]:
def HSDM_4D_likelihood(prms, RT, Theta, N_series):
    a = prms[0]
    ndt = prms[1]
    mu = np.array([prms[2], prms[3], prms[4], prms[5]])
    
    tt = np.arange(0.001, max(RT)+0.02, 0.02)
    fpt = series_bessel_fpt(tt, a, sigma=1, nu=(mu.shape[0]-2)/2, n=N_series)
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta[0])
            mu_dot_x1 = mu[1]*np.sin(theta[0])*np.cos(theta[1]) 
            mu_dot_x2 = mu[2]*np.sin(theta[0])*np.sin(theta[1])*np.cos(theta[2])
            mu_dot_x3 = mu[3]*np.sin(theta[0])*np.sin(theta[1])*np.sin(theta[2]) 
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1 + mu_dot_x2 + mu_dot_x3)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt-ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': [],
               'mu4_true': [],
               'mu4_estimate': []}

min_threshold = 0.5
max_threshold = 5

min_ndt = 0.1
max_ndt = 1

min_mu = -3.5
max_mu = 3.5

N_series = 150

In [6]:
for n in tqdm(range(150)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    recovery_df['mu3_true'].append(mu[2])
    recovery_df['mu4_true'].append(mu[3])
    
    RT = []
    Theta = []
    
    for i in range(250):
        rt, theta = simulate_HSDM_4D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = minimize(HSDM_4D_likelihood,
                       args=(RT, Theta, N_series), 
                       x0=np.array([np.random.uniform(min_threshold, max_threshold),
                                    np.random.uniform(min_ndt, max_ndt), 
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu),
                                    np.random.uniform(min_mu, max_mu)]),
                       method='Nelder-Mead', 
                       bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                               (min_mu, max_mu), (min_mu, max_mu), 
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    recovery_df['mu3_estimate'].append(min_ans.x[4])
    recovery_df['mu4_estimate'].append(min_ans.x[5])
    
recovery_df = pd.DataFrame(recovery_df)

100%|████████████████████████████| 150/150 [7:54:03<00:00, 189.62s/it]


In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate,mu4_true,mu4_estimate
0,2.898455,1.842136,0.784670,1.000000,-2.126012,-2.109096,2.918106,2.871519,1.665842,1.760913,2.038940,2.027157
1,4.330039,4.451779,0.733075,0.773567,0.458336,0.454932,0.106903,0.098655,1.496032,1.650426,-1.821801,-1.842223
2,2.421368,2.299695,0.938741,1.000000,-1.572063,-1.667691,2.717798,2.775749,-0.027704,-0.063705,0.301947,0.291445
3,1.700740,1.645758,0.619170,0.635169,1.790090,1.719039,0.857736,0.742524,-0.436388,-0.356344,1.963442,1.955692
4,0.541552,0.500000,0.786133,0.797710,-1.652045,-1.570989,-1.483257,-2.132926,0.959613,0.839794,1.630545,1.974584
...,...,...,...,...,...,...,...,...,...,...,...,...
145,3.812978,3.264594,0.832563,1.000000,-1.601825,-1.695875,-1.924692,-1.967951,-1.534590,-1.692138,-2.810779,-3.086728
146,4.286297,4.257203,0.767921,0.794576,0.872123,0.860368,3.437355,3.500000,2.347968,2.270319,-1.373841,-1.560942
147,4.409537,4.417965,0.903180,0.892957,1.134032,1.160386,1.454566,1.509994,3.478622,3.498854,-2.990929,-2.903023
148,1.305952,1.184214,0.727414,0.754394,1.383199,3.272533,-0.875338,-0.773912,1.215176,0.981552,-3.224158,-2.125009


In [8]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate,mu4_true,mu4_estimate
threshold_true,1.000000,0.792434,-0.005031,0.018185,-0.047906,0.022868,-0.083296,-0.025105,-0.019218,0.022554,-0.071162,-0.055584
threshold_estimate,0.792434,1.000000,-0.082178,-0.069455,-0.007877,0.040122,-0.063017,-0.033081,0.082496,0.044196,-0.079836,-0.097057
ndt_true,-0.005031,-0.082178,1.000000,0.822377,-0.170229,-0.139908,0.157259,0.148586,-0.062468,-0.011750,-0.085007,-0.127185
ndt_estimate,0.018185,-0.069455,0.822377,1.000000,-0.080860,-0.098191,0.155072,0.176030,-0.016476,-0.044344,-0.120174,-0.094812
mu1_true,-0.047906,-0.007877,-0.170229,-0.080860,1.000000,0.837963,-0.066779,-0.101772,0.086446,0.029199,-0.051748,-0.054773
mu1_estimate,0.022868,0.040122,-0.139908,-0.098191,0.837963,1.000000,-0.007947,-0.007890,0.024384,0.124958,-0.062050,-0.043951
mu2_true,-0.083296,-0.063017,0.157259,0.155072,-0.066779,-0.007947,1.000000,0.931624,0.107323,0.099821,0.059543,-0.004360
mu2_estimate,-0.025105,-0.033081,0.148586,0.176030,-0.101772,-0.007890,0.931624,1.000000,0.064236,0.052966,0.008970,0.019060
mu3_true,-0.019218,0.082496,-0.062468,-0.016476,0.086446,0.024384,0.107323,0.064236,1.000000,0.848121,-0.096994,-0.108918
mu3_estimate,0.022554,0.044196,-0.011750,-0.044344,0.029199,0.124958,0.099821,0.052966,0.848121,1.000000,-0.090421,-0.116476


In [9]:
recovery_df.to_csv('Series_4d_recovery_{}.csv'.format(N_series))